In [1]:
import sqlite3
import random
import pandas as pd

In [2]:
db = sqlite3.connect('test.db') 
c = db.cursor() 

## Start

In [3]:
level, category = 1, 1
questions_total, rating = 0, 0
prev_qs = set()
user_results = {}

Функция подбора вопроса (рандомный выбор вопроса нужного уровня из нужной категории + исключение повторения вопросов)

In [4]:
def choose_question(category, level, prev_qs):
    db_query = 'SELECT * from Questions WHERE id_level="{}" AND id_category="{}"'.format(level, category)
    q_list = [q for q in list(c.execute(db_query)) if q[0] not in prev_qs]
    question = random.choice(q_list)
    prev_qs.add(question[0])
    return question, prev_qs

Функция вывода вопроса и проверки ответа пользователя

In [5]:
def ask_question(rating, category, level, prev_qs):
    question = list(choose_question(category, level, prev_qs)[0])
    name_q = question[1]
    i = 1
    var_q = []
    for v in question[2].split('/'):
        var_q.append(str(i) + ') ' + v + '\n')  #формируем красивый список вариантов ответа
        i += 1
    ans = question[3]
    print(name_q + '\n' + ''.join(var_q))
    user_ans = int(input())
    if user_ans == ans:
        print('Ответ верный!')
        rating += 1
    else:
        print('Ответ неверный!')
        if rating > 0:
            rating = rating - 1  #за неправильный ответ понижаем рейтинг, если он не 0
    return rating

Функция проверки рейтинга и перехода на новый уровень, если рейтинг 2 

In [6]:
def check_rating(rating, level):
    if rating >= 2:
        level += 1
        print('Ура! Ты перешел на новый уровень!')
        rating = 0  #на новом уровне рейтинг обнуляется
    return level, rating

Функция определения финального результата по итогам каждой категории

In [7]:
def get_final(user_results):
    defined_results = {}
    for r in user_results:
        if user_results[r] == 1:
            defined_results[r] = 'A0 (начальный уровень)'
        elif user_results[r] == 2:
            defined_results[r] = 'A1 (элементарный уровень)'
        elif user_results[r] == 3:
            defined_results[r] = 'А2 (предпороговый уровень)'
        elif user_results[r] == 4:
            defined_results[r] = 'B1 (пороговый уровень)'
        elif user_results[r] == 5:
            defined_results[r] = 'B2 (повышенный уровень)'
        elif user_results[r] == 6:
            defined_results[r] = 'C1 (продвинутый уровень)'
    return defined_results

In [ ]:
while category <= 3:
    while questions_total < 12:
        rating = ask_question(rating, category, level, prev_qs)
        level, rating = check_rating(rating, level)
        questions_total += 1
        print('Total: ' + str(questions_total))
    print('Молодец! Ты закончил категорию' + str(category))
    if rating == 0 and level>1:
        level = level-1  #если на вопросы последнего уровня сложности пользователь вообще не ответил, то считаем, что на этот уровень он пока не дошел
    user_results[str(category)] = level  #записываем результат категории
    category += 1
    rating, questions_total = 0, 0
    level = 1

final_results = get_final(user_results)
print('Невероятно! Ты прошел этот тест! Вот твои результаты:')
print('Орфография и орфоэпия: ' + final_results['1'])
print('Морфология: ' + final_results['2'])
print('Синтаксис и словоупотребление: ' + final_results['3'])